In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os

In [136]:
df = pd.read_csv('../datasets/btcusdt_5malone.csv')
df = df[df['is_kline_closed'] == True]
df = df.reset_index(drop=True)
df.head()

,id,event_type,event_time,symbol,kline_start_time,kline_close_time,interval,first_trade_id,last_trade_id,open_price,...,high_price,low_price,volume,number_of_trades,is_kline_closed,quote_assest_volume,take_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,latency
0,66,kline,1721237700008,BTCUSDT,1721237400000,1721237699999,5m,3689084949,3689090661,64234.01,...,64319.98,64203.34,87.07039,5713,True,5.596994e+06,58.43994,3.756628e+06,0,109
1,200,kline,1721238000009,BTCUSDT,1721237700000,1721237999999,5m,3689090662,3689094509,64305.37,...,64375.00,64280.00,89.37234,3848,True,5.748167e+06,68.14521,4.382936e+06,0,106
2,331,kline,1721238300012,BTCUSDT,1721238000000,1721238299999,5m,3689094510,3689098295,64375.00,...,64411.76,64330.82,65.52775,3786,True,4.218289e+06,34.91378,2.247542e+06,0,113
3,462,kline,1721238600009,BTCUSDT,1721238300000,1721238599999,5m,3689098296,3689102259,64360.02,...,64374.00,64264.00,71.97426,3964,True,4.630275e+06,35.24121,2.267379e+06,0,133
4,593,kline,1721238900008,BTCUSDT,1721238600000,1721238899999,5m,3689102260,3689105618,64350.47,...,64367.09,64270.00,65.17340,3359,True,4.191142e+06,38.74083,2.491350e+06,0,120


In [137]:
df = df.rename(columns={
    'quote_assest_volume': 'quote_asset_volume',
    'open_price': 'open',
    'close_price': 'close',
    'low_price': 'low',
    'high_price': 'high',
    'number_of_trades': 'trades',
    'kline_start_time': 'start_time',
    'kline_close_time': 'close_time'
})

In [138]:
columns_to_drop =[
    'id',
    'event_type',
    'symbol',
    'interval',
    'first_trade_id',
    'last_trade_id',
    'is_kline_closed',
    'ignore',
    'latency'
]
df = df.drop(columns_to_drop, axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   event_time                    8 non-null      int64  
 1   start_time                    8 non-null      int64  
 2   close_time                    8 non-null      int64  
 3   open                          8 non-null      float64
 4   close                         8 non-null      float64
 5   high                          8 non-null      float64
 6   low                           8 non-null      float64
 7   volume                        8 non-null      float64
 8   trades                        8 non-null      int64  
 9   quote_asset_volume            8 non-null      float64
 10  take_buy_base_asset_volume    8 non-null      float64
 11  taker_buy_quote_asset_volume  8 non-null      float64
dtypes: float64(8), int64(4)
memory usage: 900.0 bytes


In [127]:
#sorting
df = df.sort_values(by='event_time')

In [128]:
# check duplicates
df[df['event_time'].duplicated(keep=False)]

In [130]:
# convert Unix timestamp into datetime
df['event_time'] = pd.to_datetime(df['event_time'], unit='ms')
df['start_time'] = pd.to_datetime(df['start_time'], unit='ms')
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')

In [131]:
# check missing values
def check_missing_rows(df, interval='5T'):
    
    min_time = df['start_time'].min()
    max_time = df['start_time'].max()
    
    
    expected_times = pd.date_range(start=min_time, end=max_time, freq=interval)
    
    
    actual_times = df['start_time']
    missing_times = expected_times.difference(actual_times)
    
    return missing_times

missing_times = check_missing_rows(df)
print("Missing times:")
missing_times

Missing times:


C:\Users\HP\AppData\Local\Temp\ipykernel_8536\743610105.py:8: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  expected_times = pd.date_range(start=min_time, end=max_time, freq=interval)


DatetimeIndex([], dtype='datetime64[ns]', freq='5min')

In [133]:
# processing before database table
df['event_time'] = df['event_time'].view('int64') // 10**6
df['start_time'] = df['start_time'].view('int64') // 10**6
df['close_time'] = df['close_time'].view('int64') // 10**6
df['volume'] = df['volume'].round(2)
df['quote_asset_volume'] = df['quote_asset_volume'].round(2)
df['take_buy_base_asset_volume'] = df['take_buy_base_asset_volume'].round(2)
df['taker_buy_quote_asset_volume'] = df['taker_buy_quote_asset_volume'].round(2)
df.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_8536\1041664752.py:2: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  df['event_time'] = df['event_time'].view('int64') // 10**6
C:\Users\HP\AppData\Local\Temp\ipykernel_8536\1041664752.py:3: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  df['start_time'] = df['start_time'].view('int64') // 10**6
C:\Users\HP\AppData\Local\Temp\ipykernel_8536\1041664752.py:4: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  df['close_time'] = df['close_time'].view('int64') // 10**6


,event_time,start_time,close_time,open,close,high,low,volume,trades,quote_asset_volume,take_buy_base_asset_volume,taker_buy_quote_asset_volume
0,1720902000010,1720901700000,1720901999999,58583.21,58593.06,58601.54,58577.28,16.60,1767,972533.13,10.42,610723.53
1,1720902300009,1720902000000,1720902299999,58593.05,58584.00,58625.97,58582.00,11.22,1716,657460.82,5.55,325416.42
2,1720902600017,1720902300000,1720902599999,58584.00,58592.00,58592.00,58574.32,25.67,1242,1503775.83,9.30,544534.60
3,1720902900009,1720902600000,1720902899999,58592.00,58573.55,58592.00,58565.18,11.89,1247,696385.41,5.64,330647.19
4,1720903200009,1720902900000,1720903199999,58573.54,58600.00,58605.48,58573.44,13.84,1347,810713.11,9.75,570933.03


#### insert into database

In [134]:
class Config:
    def __init__(self):
        self.host = os.getenv('host')
        self.db_name = os.getenv('database')
        self.db_user = os.getenv('user')
        self.db_password = os.getenv('password')

config = Config()

host = os.getenv('host')
db_name = os.getenv('database')
db_user = os.getenv('user')
db_password = os.getenv('password')

In [135]:
connection_string = f'postgresql://{db_user}:{db_password}@{host}/{db_name}'

engine = create_engine(connection_string)

table_name = 'btcusdt_5m_demo'

df.to_sql(table_name, engine, if_exists='replace', index=False)

print(f"DataFrame has been successfully inserted into the table '{table_name}' in the database.")

DataFrame has been successfully inserted into the table 'btcusdt_5m_demo' in the database.
